In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns
%matplotlib inline

In [2]:
dt= pd.read_csv('Algerian_forest_fires_dataset_Cleaned.csv')

In [3]:
dt.head()

,day,month,year,Temperature,RH,Ws,Rain,FFMC,DMC,DC,ISI,BUI,FWI,Classes,Region
0,1,6,2012,29,57,18,0.0,65.7,3.4,7.6,1.3,3.4,0.5,0,0
1,2,6,2012,29,61,13,1.3,64.4,4.1,7.6,1.0,3.9,0.4,0,0
2,3,6,2012,26,82,22,13.1,47.1,2.5,7.1,0.3,2.7,0.1,0,0
3,4,6,2012,25,89,13,2.5,28.6,1.3,6.9,0.0,1.7,0.0,0,0
4,5,6,2012,27,77,16,0.0,64.8,3.0,14.2,1.2,3.9,0.5,0,0


In [4]:
dt.drop(['day','month','year'],axis=1,inplace=True)

Predict FWI

In [5]:
X=dt.drop('FWI', axis=1)
y=dt['FWI']

In [6]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test= train_test_split(X,y,train_size=0.25,random_state=42)

In [7]:
##Feature Selection based on correlation
x_train.corr()

,Temperature,RH,Ws,Rain,FFMC,DMC,DC,ISI,BUI,Classes,Region
Temperature,1.000000,-0.630008,-0.451991,-0.347665,0.734939,0.594049,0.424107,0.666561,0.548067,0.619375,0.356735
RH,-0.630008,1.000000,0.299674,0.227513,-0.648896,-0.334133,-0.151579,-0.763708,-0.271276,-0.434327,-0.492784
Ws,-0.451991,0.299674,1.000000,0.191249,-0.215066,-0.103543,0.006133,-0.170100,-0.062355,-0.134433,-0.251231
Rain,-0.347665,0.227513,0.191249,1.000000,-0.592178,-0.365088,-0.381958,-0.418883,-0.380219,-0.474113,-0.100790
FFMC,0.734939,-0.648896,-0.215066,-0.592178,1.000000,0.651893,0.579686,0.777809,0.643711,0.801885,0.287874
DMC,0.594049,-0.334133,-0.103543,-0.365088,0.651893,1.000000,0.902410,0.647746,0.986077,0.702528,0.232512
DC,0.424107,-0.151579,0.006133,-0.381958,0.579686,0.902410,1.000000,0.488922,0.959562,0.650110,-0.026547
ISI,0.666561,-0.763708,-0.170100,-0.418883,0.777809,0.647746,0.488922,1.000000,0.603473,0.767419,0.373099
BUI,0.548067,-0.271276,-0.062355,-0.380219,0.643711,0.986077,0.959562,0.603473,1.000000,0.703227,0.136544
Classes,0.619375,-0.434327,-0.134433,-0.474113,0.801885,0.702528,0.650110,0.767419,0.703227,1.000000,0.231755


In [8]:
#check for multicollinearlity
corr=x_train.corr()
sns.heatmap(corr,annot=True)

<Axes: >

In [9]:
def correlation(dataset,threshold):
    col_corr= set()
    corr_matrix = dataset.corr()
    for i in range(len(corr_matrix.columns)):
        for j in range(i):
            if abs(corr_matrix.iloc[i,j])>threshold:
                colname = corr_matrix.columns[i]
                col_corr.add(colname)
    return col_corr

In [10]:
corr_features = correlation(x_train,0.85)
corr_features

{'BUI', 'DC'}

In [11]:
x_train.drop(corr_features,axis=1,inplace=True)
x_test.drop(corr_features,axis=1,inplace=True)

In [12]:
from sklearn.preprocessing import StandardScaler
sc= StandardScaler()
X_train_sc=sc.fit_transform(x_train)
X_test_sc=sc.transform(x_test)

In [13]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(X_train_sc,y_train)
y_pred=model.predict(X_test_sc)


In [14]:
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error
score= r2_score(y_test,y_pred)
mae = mean_absolute_error(y_test,y_pred)
mse= mean_squared_error(y_test,y_pred)
print(score, mae, mse)

0.9639181608365198 0.7729775755962681 1.9743632944114493


In [15]:
plt.scatter(y_test,y_pred)

AttributeError: module 'matplotlib' has no attribute 'scatter'

Lasso regression

In [16]:
from sklearn.linear_model import Lasso 
lasso = Lasso()
lasso.fit(X_train_sc,y_train)
y_pred= lasso.predict(X_test_sc)

In [18]:
scoreLasso = r2_score(y_test,y_pred)
mae= mean_absolute_error(y_test,y_pred)
print(scoreLasso, mae)

0.9306206026377476 1.3995835920782733


In [ ]:
from sklearn.linear_model import LassoCV
model=LassoCV(cv=5)
model.fit(X_train_sc,y_train)

LassoCV(cv=5)

In [25]:
y_pred = model.predict(X_test_sc)
scoreLasso = r2_score(y_test,y_pred)
mae= mean_absolute_error(y_test,y_pred)
print(scoreLasso, mae)

0.9639181608365198 0.7729775755962681


Ridge Regessoion

In [33]:
from sklearn.linear_model import Ridge 
ridge = Ridge()
ridge.fit(X_train_sc,y_train)
y_pred = ridge.predict(X_test_sc)
score = r2_score(y_test,y_pred)
mae= mean_absolute_error(y_test,y_pred)
print(score, mae)

0.9626401499678693 0.8618428795141457


In [35]:
from sklearn.linear_model import RidgeCV
ridgecv = RidgeCV(cv=5)
ridgecv.fit(X_train_sc,y_train)
y_pred = ridgecv.predict(X_test_sc)
score = r2_score(y_test,y_pred)
mae= mean_absolute_error(y_test,y_pred)
print(score, mae)

0.9638694844551805 0.7819344224003705


In [30]:
modelcv.get_params()

{'alpha_per_target': False,
 'alphas': (0.1, 1.0, 10.0),
 'cv': 5,
 'fit_intercept': True,
 'gcv_mode': None,
 'scoring': None,
 'store_cv_results': None,
 'store_cv_values': 'deprecated'}

Elastic regression

In [32]:
from sklearn.linear_model import ElasticNet, ElasticNetCV
model1 = ElasticNet()
model2= ElasticNetCV(cv=5)

model1.fit(X_train_sc,y_train)
model2.fit(X_train_sc,y_train)

y_pred1= model1.predict(X_test_sc)
y_pred2= model2.predict(X_test_sc)

score1 = r2_score(y_test, y_pred1)
score2 = r2_score(y_test, y_pred2)

print(score1,score2)

0.8462459371489847 0.9636233633245219


In [36]:
##pickle machine learning models, preprocessing model standardscalar
ridgecv

RidgeCV(cv=5)

In [37]:
sc

StandardScaler()

In [38]:
import pickle
pickle.dump(sc,open('scaler.pkl','wb'))
pickle.dump(ridgecv,open('ridge.pkl','wb'))